In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import json
from sklearn import linear_model
import chardet
import gzip
import tarfile
import matplotlib.pyplot as plt
from collections import defaultdict
import statistics
from gensim.models import Word2Vec
from nltk.stem.porter import *
from sklearn import linear_model
import string
import nltk
import random
from nltk.corpus import stopwords
import datetime
from dateutil import parser
#from fastFM import als

In [2]:
#conda install -c conda-forge fastFM

In [3]:
def readJSON(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        d = eval(l)
        yield d

In [4]:
def parseData(fname):
    with open(fname, 'r') as file:
        for line in file:
            yield json.loads(line)

In [ ]:
f = gzip.open("movies.txt.gz", 'rt')
content = f.read()
#dataset = list(read("movies.txt.gz"))

In [ ]:
len(dataset)

In [ ]:
dataset[0], dataset[1]

In [ ]:
# Assuming 'dataset' is your list of dictionaries
max_unix_review_time = max(dataset, key=lambda x: x['unixReviewTime'])['unixReviewTime']

print("Max Unix Review Time:", max_unix_review_time)

In [ ]:
# Assuming 'dataset' is your list of dictionaries
min_unix_review_time = min(dataset, key=lambda x: x['unixReviewTime'])['unixReviewTime']

print("Min Unix Review Time:", min_unix_review_time)

In [ ]:
#Pandas Conversion

In [ ]:
df = pd.DataFrame(dataset)

In [ ]:
df.info()

Never touch the above code

In [ ]:
df.shape

In [ ]:
df['']

In [ ]:
sns.set(style='whitegrid', palette="deep", font_scale=1.1, rc={"figure.figsize": [8, 5]})

In [ ]:
sns.distplot(
    df['overall'], norm_hist=False, kde=False, bins=50, hist_kws={"alpha": 1}
).set(xlabel='Overall Rating', ylabel='Count');

In [ ]:
numerical = ['overall','unixReviewTime']
categorical = ['reviewerName','asin']

In [ ]:
asg = df.groupby('asin')
usg = df.groupby('reviewerID')

In [ ]:
#sns.countplot(df.groupby('asin'))

In [ ]:
asg.get_group('0005089549')

In [ ]:
usg.groups

In [ ]:
usg.get_group('A2M1CU2IRZG0K9')

In [ ]:
suml = [a for a in dataset if a['reviewerID'] == 'A2M1CU2IRZG0K9']
#suml

In [ ]:
suml

In [ ]:
dataset2 = list(parseData("Electronics_5.json"))

In [ ]:
len(dataset2)

In [ ]:
dataset2[:5]

In [ ]:
dataset[100000]

In [ ]:
nltk.download('stopwords')

Movie Recommendation

In [ ]:
#Popular movie reviews
rev_count = defaultdict(list)
for a in dataset:
    if a['asin'] in rev_count: rev_count[a['asin']] += 1
    else: rev_count[a['asin']] = 1

In [ ]:
rev_count['6300214338']

In [ ]:
min(rev_count.values()), max(rev_count.values())

In [ ]:
val_arr = list(rev_count.values())

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x = val_arr)
plt.xlabel('Number of Reviews')
plt.show()

In [ ]:
pop_movies = [b for b in rev_count if rev_count[b] > 50]

In [ ]:
len(pop_movies)

In [ ]:
pop_ds = [d for d in dataset if rev_count[d['asin']] > 1000]

In [ ]:
len(pop_ds)

In [ ]:
sum(k['overall']==1.0 for k in pop_ds)

In [ ]:
sum(k['overall']==2.0 for k in pop_ds)

In [ ]:
sum(k['overall']==3.0 for k in pop_ds)

In [ ]:
sum(k['overall']==4.0 for k in pop_ds)

In [ ]:
sum(k['overall']==5.0 for k in pop_ds)

In [ ]:
#Ratio of 1,5,2 with 2M total entries

116583/2235675, 1347585/2235675, 107736/2235675

In [ ]:
#Ratio of 1,5,2 with 638K entries

30041/638665, 389843/638665, 29198/638665

In [ ]:
pop_ds[1000]

In [ ]:
#Scatter plot overall vs review time
review_ratings = [entry['overall'] for entry in pop_ds]
review_times = [entry['unixReviewTime'] for entry in pop_ds]

# Create a scatter plot using Seaborn
plt.figure(figsize=(10, 6))
sns.scatterplot(x=review_times, y=review_ratings, alpha=1.0)  # Adding alpha for transparency

# Adding labels and title
plt.title('Review Rating vs. Unix Review Time for All Movies')
plt.xlabel('Unix Review Time')
plt.ylabel('Review Rating')

# Display the plot
plt.show()

In [ ]:
#Linear line plot Time vs Rating
plt.figure(figsize=(10, 6))
sns.lineplot(x=review_times, y=review_ratings, ci=None) 

# Adding labels and title
plt.title('Review Rating vs. Unix Review Time for All Movies')
plt.xlabel('Unix Review Time')
plt.ylabel('Review Rating')

# Display the plot
plt.show()

In [ ]:
selected_asin = '0782010792'

# Filter the dataset for the specific asin
selected_data = [entry for entry in pop_ds if entry['asin'] == selected_asin]

# Extract relevant data for the plot
review_ratings = [entry['overall'] for entry in selected_data]
review_times = [entry['unixReviewTime'] for entry in selected_data]

# Create a scatter plot using Seaborn
plt.figure(figsize=(10, 6))
sns.lineplot(x=review_times, y=review_ratings)

# Adding labels and title
plt.title('Review Rating vs. Unix Review Time')
plt.xlabel('Unix Review Time')
plt.ylabel('Review Rating')

# Display the plot
plt.show()

In [ ]:
pop_df = pd.DataFrame(pop_ds)

# Convert Unix Review Time to datetime
pop_df['review_date'] = pd.to_datetime(pop_df['unixReviewTime'], unit='s')

# Create a histogram using Seaborn
plt.figure(figsize=(12, 6))
sns.histplot(data=pop_df, x='review_date', y='overall', bins='auto', cbar=True, cmap='viridis')

# Adding labels and title
plt.title('Distribution of Review Ratings over Time for All Movies')
plt.xlabel('Review Date')
plt.ylabel('Review Rating')

# Display the plot
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.tight_layout()  # Adjust layout for better visualization
plt.show()

In [ ]:
one = [k for k in pop_ds if k['overall']==1.0]

In [ ]:
len(one)

In [ ]:
one[100]

In [ ]:
selected_asin = '0790729628'

# Convert list of dictionaries to a pandas DataFrame
pop_df = pd.DataFrame(pop_ds)

# Filter the dataset for the specific asin
selected_data = pop_df[pop_df['asin'] == selected_asin]

# Convert Unix Review Time to datetime and extract the date
selected_data['review_date'] = pd.to_datetime(selected_data['unixReviewTime'], unit='s').dt.date

# Create a simple line plot using Matplotlib
plt.figure(figsize=(12, 6))
plt.plot(selected_data['review_date'], selected_data['overall'], color='b')

# Adding labels and title
plt.title(f'Review Ratings over Time for Movie {selected_asin}')
plt.xlabel('Review Date')
plt.ylabel('Review Rating')

# Display the plot
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.tight_layout()  # Adjust layout for better visualization
plt.show()

In [ ]:
len(selected_data)

In [ ]:
pop_ds[700]

In [ ]:
a = []
for d in pop_ds:
    review_text = d.get('reviewText', '')  # Use get to handle missing key
    a.append(review_text)

In [ ]:
len(a), len(pop_ds)

In [ ]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
stemmer = PorterStemmer()
for d in dataset:
    r = ''.join([c.lower() for c in a if not c in punctuation])
    for w in r.split():
        w = stemmer.stem(w)
        wordCount[w] += 1

In [ ]:
len(wordCount)

In [ ]:
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [ ]:
counts[:10]

In [ ]:
#Fresh Implementation

In [ ]:
#Creating a list of popular movie with more than 2k reviews
pop_movies = [d for d in dataset if rev_count[d['asin']] > 2000]
len(pop_movies)

In [ ]:
pop_movies[10009]

In [ ]:
#pop_movies = random.shuffle(pop_movies)

In [ ]:
#Fetch the reviews of the popular movies
review_text = [d.get('reviewText') for d in pop_movies[:170000]]

In [ ]:
#Creating Stop words list
StopWords = stopwords.words("english")

In [ ]:
len(StopWords)

In [ ]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
stemmer = PorterStemmer()
for w in review_text:
    if w:
        #Being case-insensitive & removing punctuations
        r = ''.join([c for c in w.lower() if not c in punctuation])
        ws = r.split()
        for k in ws:
            #Removing stop words in a review
            if k not in StopWords:
                #Fetching Stem words in a review
                k = stemmer.stem(k)
                wordCount[k] += 1

len(wordCount)

In [ ]:
#Count of the most used words
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [ ]:
counts[:10]

In [ ]:
#Bases on word count creating unique word set
words = [x[1] for x in counts[:5000]]
len(words)

In [ ]:
#Assigning weights to words
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [ ]:
#Defining feature vector
def feature(datum):
    feat = [0]*len(words)
    review_text_2 = datum.get('reviewText')
    if review_text_2:
        r = ''.join([c for c in review_text_2.lower() if not c in punctuation])
        ws = r.split()
        for w in ws:
            w = stemmer.stem(w)
            if w in words:
                feat[wordId[w]] += 1
    feat.append(1) #offset
    return feat

In [ ]:
X_train = [feature(d) for d in pop_movies[:170000]]

In [ ]:
y_train = [d['overall'] for d in pop_movies[:170000]]

In [ ]:
X_test = [feature(d) for d in pop_movies[170000:]]

In [ ]:
y_test = [d['overall'] for d in pop_movies[170000:]]

In [ ]:
clf = linear_model.Ridge(1.0, fit_intercept=False) # MSE + 1.0 l2
clf.fit(X_train, y_train)
theta = clf.coef_
predictions = clf.predict(X_test)

In [ ]:
# Failed FastFM trial
#fm = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=5, l2_reg_w=0.1, l2_reg_V=0.5)
# fm.fit(X_train, y_train)

In [ ]:
wordSort = list(zip(theta[:-1], words))
wordSort.sort()

In [ ]:
#Feature 2 - Scaled Review Length
rev_len = [len(d.get('reviewText')) for d in pop_movies[:170000] if d.get('reviewText')]
rev_len = [(a-min(rev_len))/max(rev_len) for a in rev_len]

In [ ]:
min(rev_len), max(rev_len)

In [ ]:
#Feature 3 - Avg rating of movies

In [ ]:
users = set()       # User-id mapping
movies = set()       # Movie-id mapping
usersPerMovie = defaultdict(set) # Maps a movie to the users who rated it
moviesPerUser = defaultdict(set) # Maps a user to the movies that they rated

for d in pop_movies[:170000]:
    u, m = d['reviewerID'], d['asin']
    usersPerMovie[m].add(u)
    moviesPerUser[u].add(m)
    if u not in users:
        users.add(u)
    if m not in movies:
        movies.add(m)

In [ ]:
len(users), len(movies)

In [ ]:
len(usersPerMovie['6305837325']), len(moviesPerUser['A22ZN5CJ3YQHVC'])

In [ ]:
avg_mov = defaultdict(list)
for m in movies:
    for d in pop_movies[:170000]:
        if d['asin']== m:
            avg_mov[m].append(d['overall']) 

for m in avg_mov:
    avg_mov[m] = sum(avg_mov[m]) / len(avg_mov[m])

In [ ]:
avg_mov['6305837325']

In [ ]:
#Feature 4 - Age of a review

In [ ]:
#First review date - December 1, 1997
#Last review date - September 30, 2018
#Consideration of date - Jan 1, 2019
#Max age - 7702 days
#Min age - 94 days
date_string = "Jan 1, 2019"
curr_dt = parser.parse(date_string)

In [ ]:
#Age of a review
age_rev = list()

for d in pop_movies[:170000]:
    dt = datetime.datetime.fromtimestamp(d['unixReviewTime'])
    time_difference = curr_dt - dt
    days_difference = time_difference.total_seconds() / (60 * 60 * 24)
    age_rev.append(days_difference) 